In [1]:
# Standard libraries
import h5py
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path1p, "lib")
pwd_mat = os.path.join(os.path.join(path2p, "data/"), "sim_ds_mat")
pwd_h5 = os.path.join(os.path.join(path2p, "data/"), "sim_ds_h5")

sys.path.insert(0, libpath)
print("Appended library directory", libpath)

# User libraries
from matlab.matlab_yaro_lib import read_mat
from signal_lib import downsample
from corr_lib import sprMat
from idtxl_wrapper import idtxlParallelCPUMulti, idtxlResultsParse
from qt_wrapper import gui_fpath

# IDTxl libraries
from idtxl.bivariate_mi import BivariateMI
from idtxl.multivariate_mi import MultivariateMI
from idtxl.bivariate_te import BivariateTE
from idtxl.multivariate_te import MultivariateTE
from idtxl.data import Data
from idtxl.visualise_graph import plot_network

Appended library directory /home/ubuntu/work/nest-mesoscopic-metric-test/codes/lib


## 1. Load Neuronal Data

The neuronal data for a given experiment is given by the 3D matrix of dimensions `[N_Trial, N_Times, N_Channels]`

Notes:
* `DT = 0.05` : the sampling interval is always 50ms
* `N_Times = 201` : the total experiment time is always 10s. All experiments are aligned

In [4]:
# Declare constants
params = {
    "exp_timestep" : 0.05, # 50ms, the standard measurement interval
    "bin_timestep" : 0.2,  # 200ms, the binned interval
    
    # The standard timesteps for different scenarios
    "range_CUE" : (1.0, 1.5),    # 1-1.5 seconds trial time
    "range_TEX" : (2.0, 3.5),    # 2-3.5 seconds trial time
    "range_LIK" : (3.5, 6.0),    # 3.5-6 seconds trial time
    "range_ALL" : (0.0, 10.0),   # 0-10  seconds

#     "samples_window" : "ALL",
    "trial_types"    : ["iGO", "iNOGO"],
    "subsampling"    : True,   # To bin or not to bin data
#     "subsampling_windows_lenghts" : 4*np.ones((1,51)),  #uniform subsampling, 50 bins
    
    "te_delay_min" : 1,
    "te_delay_max" : 5
    
    
    
#     "te_delay_stepsize" : 1,
#     "te_receiver_embedding_tau" : 1,
#     "te_sampling_rate" : 20
}

# GUI: Select videos for training
pwd_tmp = "./"
next_path = "AAAAA"
params['pwd_data_folders'] = []
while next_path != '':
    print("Currently using data folders:", params['pwd_data_folders'])
    next_path = gui_fpath("Select one more data directory, or cancel", pwd_tmp)
    if next_path != '':
        params['pwd_data_folders'] += [next_path]
        pwd_tmp = os.path.dirname(next_path)  # Next time choose from parent folder
print('Done')

Currently using data folders: []
Currently using data folders: ['/mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_10_a']
Done


## Swipe Transfer Entropy via IDTxl

Notation of IDTxl
* `(processes, samples, replications)` equivalent to ours `(channels, times, trials)`
* Thus Yaro matrix `(nTrials, nTimes, nChannels)` is given by `'rsp'`, but
* For example `(nChannels, nTimes)` would be `'ps'`

In [ ]:
%%time
for folderPathName in params['pwd_data_folders']:
    folderName = os.path.basename(folderPathName)
    
    #############################
    # Reading and downsampling
    #############################

    # Read LVM file from command line
    data, behaviour = read_mat(folderPathName)

    # Get parameters
    nTrials, nTimes, nChannels = data.shape
    print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)

    assert nTimes == 201, "The number of timesteps must be 201 for all data, got "+str(nTimes)
    nTimesDownsampled = int(nTimes * params["exp_timestep"] / params["bin_timestep"])
    print(nTimesDownsampled)

    # Timeline (x-axis)
    tlst = params["exp_timestep"] * np.linspace(0, nTimes, nTimes)

    # Downsample data
    print("Downsampling from", params["exp_timestep"], "ms to", params["bin_timestep"], "ms")
    if params["subsampling"]:
        params["timestep"] = params["bin_timestep"]
        tlst_new = downsample(tlst, data[0, :, 0], nTimesDownsampled)[0]
        data_new = np.array([[downsample(tlst, data[i, :, j], nTimesDownsampled)[1] 
                              for i in range(nTrials)] 
                             for j in range(nChannels)])
    else:
        params["timestep"] = params["exp_timestep"]

#     # Plot downsampling for comparison
#     plt.figure()
#     plt.plot(tlst, data[0, :, 0], label='original')
#     plt.plot(tlst_new, data_new[0, 0, :], '.-', label='downsampled')
#     plt.title("Example before and after downsampling")
#     plt.legend()
#     plt.show()

    # Replace old data with subsampled one    
    tlst, data = tlst_new, data_new.transpose(1, 2, 0)
    nTrials, nTimes, nChannels = data.shape
    print("After downsampling data shape is (nTrials, nTimes, nChannels)=", data.shape)
    
    for trialType in params['trial_types']:
        
        if trialType is None:
            dataEff = data
            nChannelsEff = nChannels
            fileNameSuffix = ""
        else:
            dataEff = data[behaviour[trialType][0] - 1]
            nChannelsEff = dataEff.shape[0]
            fileNameSuffix = "_" + trialType
            print("For trialType =", trialType, "the shape is (nTrials, nTimes, nChannels)=", dataEff.shape)

        #############################
        # Analysis
        #############################

        idtxl_settings = {
            'dim_order'       : 'rsp',
            'methods'         : ["BivariateTE", "MultivariateTE"],
            'cmi_estimator'   : 'JidtGaussianCMI',
            'max_lag_sources' : params["te_delay_max"],
            'min_lag_sources' : params["te_delay_min"]
       }

        teWindow = params["te_delay_max"] + 1

        teSwipeMat  = np.zeros((nChannelsEff, nChannelsEff, nTimes)) + np.nan
        lagSwipeMat = np.zeros((nChannelsEff, nChannelsEff, nTimes)) + np.nan
        pSwipeMat   = np.zeros((nChannelsEff, nChannelsEff, nTimes)) + np.nan

        data_range = list(range(nTimes - teWindow))
        data_lst = [dataEff[:, i:i + teWindow, :] for i in data_range]
        rez = idtxlParallelCPUMulti(data_lst, idtxl_settings, folderName)

        for iMethod, method in enumerate(idtxl_settings['methods']):
            for iRange in data_range:
                te_mat, lag_mat, p_mat = idtxlResultsParse(rez[iMethod][iRange], nChannelsEff, method=method, storage='matrix')

                teSwipeMat[:,:, iRange + params["te_delay_max"]]  = te_mat
                lagSwipeMat[:,:, iRange + params["te_delay_max"]] = lag_mat
                pSwipeMat[:,:, iRange + params["te_delay_max"]]   = p_mat

            #######################
            # Save results to file
            #######################
            savename = os.path.join(pwd_h5, folderName + fileNameSuffix + '_' + method + '_swipe' + '.h5')
            print(savename)

            h5f = h5py.File(savename, "w")

            grp_rez = h5f.create_group("results")
            grp_rez['TE_table'] = te_mat
            grp_rez['delay_table'] = lag_mat
            grp_rez['p_table'] = p_mat

            h5f.close()

Reading Yaro data from /mnt/hifo-scratch-01/Yaro/data_for_TE/mvg_48/mvg_4/mvg_4_2017_11_10_a
Loaded neuronal data with (nTrials, nTimes, nChannels)= (1000, 201, 48)
50
Downsampling from 0.05 ms to 0.2 ms
After downsampling data shape is (nTrials, nTimes, nChannels)= (1000, 50, 48)
For trialType = iGO the shape is (nTrials, nTimes, nChannels)= (176, 50, 48)
Adding data with properties: 48 processes, 6 samples, 176 replications
overwriting existing data
Adding data with properties: 48 processes, 6 samples, 176 replications
overwriting existing data
Adding data with properties: 48 processes, 6 samples, 176 replications
overwriting existing data
Adding data with properties: 48 processes, 6 samples, 176 replications
overwriting existing data
Adding data with properties: 48 processes, 6 samples, 176 replications
overwriting existing data
Adding data with properties: 48 processes, 6 samples, 176 replications
overwriting existing data
Adding data with properties: 48 processes, 6 samples, 176 r

Target: 27 - testing sources [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

---------------------------- (1) include target candidates
candidate set: [(27, 1), (27, 2), (27, 3), (27, 4), (27, 5)]
testing candidate: (27, 1) maximum statistic, n_perm: 200

Target: 14 - testing sources [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]

---------------------------- (1) include target candidates
candidate set: [(14, 1), (14, 2), (14, 3), (14, 4), (14, 5)]
testing candidate: (0, 2) maximum statistic, n_perm: 200
testing candidate: (14, 1) maximum statistic, n_perm: 200
testing candidate: (35, 5) maximum statistic, n_perm: 200
testing candidate: (22, 5) maximum statistic, n_perm: 200

Target: 1 - testing sources [0, 2, 3, 4, 5, 6, 7, 8

testing candidate: (6, 2) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (2) include source candidates
candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
testing candidate: (0, 1) maximum statistic, n_perm: 200
testing candidate: (40, 3) maximum statistic, n_perm: 200

Target: 11 - testing sources [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]testing candidate: (19, 4) maximum statistic, n_perm: 200


---------------------------- (1) include target candidates
candidate set: [(11, 1), (11, 2), (11, 3), (11, 4), (11, 5)]
testing candidate: (11, 1) maximum statistic, n_perm: 200
testing candidate: (41, 2) maximum statistic, n_perm: 200
testing candidate: (1, 4) maximum statistic, n_perm: 200

Target: 46 - testing sources [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23,

testing candidate: (0, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
 -- not significant
testing candidate: (1, 3) maximum statistic, n_perm: 200

---------------------------- (2) include source candidates
candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
testing candidate: (0, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 4) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (2) include source candidates
candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
testing candidate: (0, 1) maximum statistic, n_perm: 200
 -- not significant
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 1) maximum statistic, n_pe

candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
testing candidate: (4, 3) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (2) include source candidates
candidate set current source: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
testing candidate: (0, 3) maximum statistic, n_perm: 200
 -- not significant
 -- not significant
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
candidate set current source: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]
testing candidate: (1, 5) maximum statistic, n_perm: 200
testing candidate: (1, 1) maximum statistic, n_perm: 200
testing candidate: (0, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
testing candidate: (1, 4) maximum statistic, n_perm: 200
testing candidate: (3, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
 -- not signifi

...

testing candidate: (28, 2) maximum statistic, n_perm: 200
candidate set current source: [(24, 1), (24, 2), (24, 3), (24, 4), (24, 5)]
testing candidate: (24, 1) maximum statistic, n_perm: 200
testing candidate: (27, 2) maximum statistic, n_perm: 200
testing candidate: (25, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(32, 1), (32, 2), (32, 3), (32, 4), (32, 5)]
testing candidate: (32, 4) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(24, 1), (24, 2), (24, 3), (24, 4), (24, 5)]
 -- not significant
 -- not significant
testing candidate: (24, 3) maximum statistic, n_perm: 200
candidate set current source: [(34, 1), (34, 2), (34, 3), (34, 4), (34, 5)]
candidate set current source: [(32, 1), (32, 2), (32, 3), (32, 4), (32, 5)]
testing candidate: (32, 2) maximum statistic, n_perm: 200
testing candidate: (34, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(38, 1), (38, 2), (38, 3)

candidate set current source: [(36, 1), (36, 2), (36, 3), (36, 4), (36, 5)]
testing candidate: (36, 4) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(31, 1), (31, 2), (31, 3), (31, 4), (31, 5)]
testing candidate: (31, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(32, 1), (32, 2), (32, 3), (32, 4), (32, 5)]
testing candidate: (32, 4) maximum statistic, n_perm: 200
 -- not significant
 -- not significant
candidate set current source: [(26, 1), (26, 2), (26, 3), (26, 4), (26, 5)]
candidate set current source: [(32, 1), (32, 2), (32, 3), (32, 4), (32, 5)]
testing candidate: (26, 1) maximum statistic, n_perm: 200
testing candidate: (32, 4) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(26, 1), (26, 2), (26, 3), (26, 4), (26, 5)]
testing candidate: (26, 3) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(23, 1), (23, 2), (23, 3), (23, 4), (23, 5)]


 -- not significant
candidate set current source: [(38, 1), (38, 2), (38, 3), (38, 4), (38, 5)]
 -- not significant
candidate set current source: [(29, 1), (29, 2), (29, 3), (29, 4), (29, 5)]
testing candidate: (38, 1) maximum statistic, n_perm: 200
testing candidate: (29, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(42, 1), (42, 2), (42, 3), (42, 4), (42, 5)]
testing candidate: (42, 4) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(34, 1), (34, 2), (34, 3), (34, 4), (34, 5)]
testing candidate: (34, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(36, 1), (36, 2), (36, 3), (36, 4), (36, 5)]
testing candidate: (36, 3) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(38, 1), (38, 2), (38, 3), (38, 4), (38, 5)]
testing candidate: (38, 4) maximum statistic, n_perm: 200
testing candidate: (22, 2) maximum statistic, n_perm: 200
 -- not significan

 -- not significant
testing candidate: (24, 3) maximum statistic, n_perm: 200
candidate set current source: [(39, 1), (39, 2), (39, 3), (39, 4), (39, 5)]
 -- not significant
candidate set current source: [(35, 1), (35, 2), (35, 3), (35, 4), (35, 5)]
testing candidate: (39, 4) maximum statistic, n_perm: 200
testing candidate: (35, 5) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(44, 1), (44, 2), (44, 3), (44, 4), (44, 5)]
testing candidate: (44, 2) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(30, 1), (30, 2), (30, 3), (30, 4), (30, 5)]
testing candidate: (30, 1) maximum statistic, n_perm: 200
testing candidate: (23, 3) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(36, 1), (36, 2), (36, 3), (36, 4), (36, 5)]
testing candidate: (36, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(31, 1), (31, 2), (31, 3), (31, 4), (31, 5)]
testing candidate:

testing candidate: (32, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(40, 1), (40, 2), (40, 3), (40, 4), (40, 5)]
 -- not significant
testing candidate: (40, 5) maximum statistic, n_perm: 200
candidate set current source: [(21, 1), (21, 2), (21, 3), (21, 4), (21, 5)]
testing candidate: (21, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(37, 1), (37, 2), (37, 3), (37, 4), (37, 5)]
testing candidate: (37, 5) maximum statistic, n_perm: 200
